In [172]:
%matplotlib inline
import re
import geonamescache
import pandas as pd

In [173]:
# regexp = re.compile("Zika")
gc = geonamescache.GeonamesCache()
from geonamescache.mappers import country
mapper = country(from_key='name', to_key='capital')
cities_by_name = gc.get_dataset_by_key(gc.get_cities(), 'name')
countries_by_name = gc.get_dataset_by_key(gc.get_countries(), 'name')

count = 0
line_count = 0
city_count = 0
city2_count = 0
city3_count = 0
country_count = 0

In [174]:
def find_city_and_country_in_word(word, prev_word, preprev_word, line_city, line_lat, line_long, line_country, line_capital, line_capital_lat,line_capital_long):
    word3 = preprev_word+' '+prev_word+' '+word
    city3 = gc.get_cities_by_name(word3)
    if city3:
        line_lat = cities_by_name[word3]['latitude'] 
        line_long = cities_by_name[word3]['longitude'] 
        line_city = word3
    else:
        word2 = prev_word+' '+word
        city2 = gc.get_cities_by_name(word2)
        if city2:
            line_lat = cities_by_name[word2]['latitude'] 
            line_long = cities_by_name[word2]['longitude'] 
            line_city = word2
        else:
            city = gc.get_cities_by_name(word)
            if city:
                line_lat = cities_by_name[word]['latitude'] 
                line_long = cities_by_name[word]['longitude'] 
                line_city = word
                
    country_capital = mapper(word)
    if country_capital:
        line_country = word
        line_capital = country_capital
        capital_city = gc.get_cities_by_name(country_capital)
        if capital_city:
            line_capital_lat = cities_by_name[country_capital]['latitude']
            line_capital_long = cities_by_name[country_capital]['longitude']
        else:
            print(word +' CAPITAL = ' +country_capital+' not found in cities dataset')
    
    return [line_city, line_lat, line_long, line_country, line_capital, line_capital_lat,line_capital_long]


In [175]:
def find_city_and_country_in_line(line, line_count):
    line_count = line_count + 1
    # init
    preprev_word = ''
    prev_word = ''
    line_city = ''
    line_country = ''
    line_capital = ''
    line_lat = 0
    line_long = 0
    line_capital_lat  = 0
    line_capital_long = 0
    # find geo names in line
    line_words = re.split('[ |?|!|\t|\n|\'|,]+', line) #'\W+'#print(line_words)
    for word in line_words:
        [line_city, line_lat, line_long, line_country, line_capital, line_capital_lat, line_capital_long] = find_city_and_country_in_word(word, prev_word, preprev_word, line_city, line_lat, line_long, line_country, line_capital, line_capital_lat, line_capital_long)
        preprev_word = prev_word
        prev_word = word
        
    return [line_city, line_lat, line_long, line_country, line_capital, line_capital_lat,line_capital_long, line_count]


In [176]:
count = 0
line_count = 0
city_count = 0
city2_count = 0
city3_count = 0
country_count = 0
line_none_count = 0
df_line = []

file = open("data/headlines.txt", 'r')
for line in file.readlines():
       
    [line_city, line_lat, line_long, line_country, line_capital, line_capital_lat, line_capital_long, line_count] = find_city_and_country_in_line(line, line_count)        
    # update names list
    line_city_name = 'NaN'
    line_country_name = 'NaN'
    if line_city:
        line_city_name = line_city
        city_count = city_count+1
        #print('Line by city: '+line_city +' '+str(line_lat))
    elif line_country:
        line_country_name = line_country
        country_count = country_count + 1
        line_lat = line_capital_lat
        line_long = line_capital_long
        #print('Line by country: '+line_country +' '+line_capital+' '+str(line_lat))
    else:
        line_none_count = line_none_count + 1
        print('Line by NONE: '+ line)
    
    df_line = df_line + [[line, line_country_name, line_city_name, line_lat, line_long]]
    

df_file = pd.DataFrame(df_line, columns = ['headline', 'countries', 'cities', 'latitude', 'longitude'])

        
file.close()
print(line_count)
# print(city_count)
# # print(city2_count)
# # print(city3_count)
# print(country_count)
print(line_none_count)

Brazil CAPITAL = Brasilia not found in cities dataset
Line by NONE: Geneve Scientists Battle to Find Cure

Line by NONE: Zika Infested Monkeys in Sao Paulo

Line by NONE: Louisiana Zika cases up to 26

Line by NONE: Zika infects pregnant woman in Cebu

Brazil CAPITAL = Brasilia not found in cities dataset
Line by NONE: 18 new Zika Cases in Bogota

Line by NONE: Spanish Flu Sighted in Antigua

Line by NONE: Carnival under threat in Rio De Janeiro due to Zika outbreak

Line by NONE: Second Zika Paitient in Brasilia

Line by NONE: Zika case reported in Oton

Line by NONE: Hillsborough uses innovative trap against Zika 20 minutes ago

Brazil CAPITAL = Brasilia not found in cities dataset
Line by NONE: Hepatitis B Vaccine is now Required in Quebec

Line by NONE: Visitor to Cucuta contracts Zika

Line by NONE: Maka City Experiences Influenza Outbreak

Line by NONE: Spreading Zika ruining Cancun fun

Line by NONE: Cancun hit by Outbreak of Party Fever!

Line by NONE: More Zika patients report

In [177]:
df_file.shape


(650, 5)

In [178]:
df_file

,headline,countries,cities,latitude,longitude
0,Zika Outbreak Hits Miami\n,NaN,Miami,25.77427,-80.19366
1,Could Zika Reach New York City?\n,NaN,New York City,40.71427,-74.00597
2,First Case of Zika in Miami Beach\n,NaN,Miami Beach,25.79065,-80.13005
3,"Mystery Virus Spreads in Recife, Brazil\n",NaN,Recife,-8.05389,-34.88111
4,Dallas man comes down with case of Zika\n,NaN,Dallas,44.91928,-123.31705
5,Trinidad confirms first Zika case\n,NaN,Trinidad,-33.51650,-56.89957
6,Zika Concerns are Spreading in Houston\n,NaN,Houston,29.76328,-95.36327
7,Geneve Scientists Battle to Find Cure\n,NaN,NaN,0.00000,0.00000
8,The CDC in Atlanta is Growing Worried\n,NaN,Atlanta,33.74900,-84.38798
9,Zika Infested Monkeys in Sao Paulo\n,NaN,NaN,0.00000,0.00000


In [169]:
word = 'Brazil'#'Istanbul'#'Louisiana'
gc.get_cities_by_name(word)

[]

In [170]:
cities_by_name[word]

KeyError: 'Brazil'

In [171]:
countries_by_name[word]

{'geonameid': 3469034,
 'name': 'Brazil',
 'iso': 'BR',
 'iso3': 'BRA',
 'isonumeric': 76,
 'fips': 'BR',
 'continentcode': 'SA',
 'capital': 'Brasilia',
 'areakm2': 8511965,
 'population': 201103330,
 'tld': '.br',
 'currencycode': 'BRL',
 'currencyname': 'Real',
 'phone': '55',
 'postalcoderegex': '^\\d{5}-\\d{3}$',
 'languages': 'pt-BR,es,en,fr',
 'neighbours': 'SR,PE,BO,UY,GY,PY,GF,VE,CO,AR'}